In [1]:
from core.adam import Adam


In [2]:
import pandas as pd
import numpy as np
from core.checker import parameters_checker
from typing import List, Union, Dict, Any
from smooth.adam_general._adam_general import adam_fitter, adam_forecaster
from core.utils.utils import measurement_inverter, scaler, calculate_likelihood, calculate_entropy, calculate_multistep_loss
from numpy.linalg import eigvals
import nlopt


In [3]:
from core.estimator import estimator, selector
from core.creator import creator, initialiser, architector, filler
from core.utils.ic import ic_function

from smooth.adam_general._adam_general import adam_fitter, adam_forecaster

import warnings

In [4]:
# Generate random monthly time series data
np.random.seed(41)  # For reproducibility
n_points = 24  # 2 years of monthly data
time_series = np.random.randint(1, 100, size=n_points).cumsum()  # Random walk with strictly positive integers
dates = pd.date_range(start='2023-01-01', periods=n_points, freq='M')  # Monthly frequency
ts_df = pd.DataFrame({'value': time_series}, index=dates)


/tmp/ipykernel_125724/4282208315.py:5: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(start='2023-01-01', periods=n_points, freq='M')  # Monthly frequency


In [5]:
%load_ext rpy2.ipython

In [6]:
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
import os

def load_smooth_dev():
    """Load the smooth package in development mode"""
    # Path to your smooth package
    smooth_path = "/home/filtheo/smooth"  # Adjust this path if needed
    
    # Load devtools and then load_all() the smooth package
    ro.r(f'''
    if (!requireNamespace("devtools", quietly=TRUE)) {{
        install.packages("devtools", repos="https://cran.rstudio.com/")
    }}
    devtools::load_all("{smooth_path}")
    ''')
    
    print("Smooth package loaded in development mode")

# Load the package in development mode
load_smooth_dev()

# Now you can access functions from the package
# For example:
# adam_function = ro.r('adam')

ℹ Loading smooth


R[write to console]: Loading required package: greybox

R[write to console]: Package "greybox", v2.0.3 loaded.


R[write to console]: This is package "smooth", v4.1.2.41003


R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
R[write to console]: 
 
R[write to console]:  library ‘/usr/lib/R/site-library’ contains no packages



Smooth package loaded in development mode


In [7]:
np.random.seed(33)  # For reproducibility
n_points = 39  # 2 years of monthly data
time_series = np.random.randint(1, 100, size=n_points).cumsum()  # Random walk with strictly positive integers
dates = pd.date_range(start='2023-01-01', periods=n_points, freq='M')  # Monthly frequency
ts_df = pd.DataFrame({'value': time_series}, index=dates)

/tmp/ipykernel_125724/2873142698.py:4: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(start='2023-01-01', periods=n_points, freq='M')  # Monthly frequency


In [8]:
%%R -i ts_df
#library(smooth)

#model <- adam(ts_df, model = "ANN", lags = c(12), print_level = 42)

#forecast(model, h = 12)




NULL


In [9]:
# Load the AirPassengers dataset from Python
import pandas as pd
import numpy as np
# Create the AirPassengers dataset manually
data = [
    112, 118, 132, 129, 121, 135, 148, 148, 136, 119, 104, 118,
    115, 126, 141, 135, 125, 149, 170, 170, 158, 133, 114, 140,
    145, 150, 178, 163, 172, 178, 199, 199, 184, 162, 146, 166,
    171, 180, 193, 181, 183, 218, 230, 242, 209, 191, 172, 194,
    196, 196, 236, 235, 229, 243, 264, 272, 237, 211, 180, 201,
    204, 188, 235, 227, 234, 264, 302, 293, 259, 229, 203, 229,
    242, 233, 267, 269, 270, 315, 364, 347, 312, 274, 237, 278,
    284, 277, 317, 313, 318, 374, 413, 405, 355, 306, 271, 306,
    315, 301, 356, 348, 355, 422, 465, 467, 404, 347, 305, 336,
    340, 318, 362, 348, 363, 435, 491, 505, 404, 359, 310, 337,
    360, 342, 406, 396, 420, 472, 548, 559, 463, 407, 362, 405,
    417, 391, 419, 461, 472, 535, 622, 606, 508, 461, 390, 432
]

# Create a proper datetime index
dates = pd.date_range(start='1949-01-01', periods=len(data), freq='MS')

# Create a pandas Series with the data
air_passengers_series = pd.Series(data, index=dates, name='AirPassengers')

# Create a DataFrame with the time series
ts_df = pd.DataFrame({'value': air_passengers_series})

print("AirPassengers dataset loaded:")
print(ts_df.head())



AirPassengers dataset loaded:
            value
1949-01-01    112
1949-02-01    118
1949-03-01    132
1949-04-01    129
1949-05-01    121


In [ ]:
%%R -i ts_df
#library(smooth)

#model <- adam(ts_df, model = "AAA", lags = c(12), h = 12)

#forecast(model, h = 12)




NULL


In [11]:
#%%R -i ts_df
#library(smooth)

##model <- adam(ts_df, model = "MAM", lags = c(12), h = 12)

#forecast(model, h = 12)


In [16]:
%%R -i ts_df
#library(smooth)

model <- adam(ts_df, model = "ZXZ", lags = c(12), h = 12)

forecast(model, h = 12)

Time Series:
Start = 145 
End = 156 
Frequency = 1 
 [1] 446.7459 421.4609 475.5442 499.7108 512.9283 585.3012 675.1290 666.5617
 [9] 553.7605 491.4647 419.3779 463.9660


In [14]:
%%R -i ts_df
#library(smooth)

model <- adam(ts_df, model = "MAM", lags = c(12), h = 12)

forecast(model, h = 12)
#model$matVt

Time Series:
Start = 145 
End = 156 
Frequency = 1 
 [1] 446.7459 421.4609 475.5442 499.7108 512.9283 585.3012 675.1290 666.5617
 [9] 553.7605 491.4647 419.3779 463.9660


In [13]:
%%R -i ts_df
#library(smooth)

model <- adam(ts_df, model = "AAdA", lags = c(12), h = 12)

forecast(model, h = 12)
#model$matVt

$lagsModel
     [,1]
[1,]    1
[2,]    1
[3,]   12

$lagsModelAll
     [,1]
[1,]    1
[2,]    1
[3,]   12

$lagsModelMax
[1] 12

$componentsNumberETS
[1] 3

$componentsNumberETSSeasonal
[1] 1

$componentsNumberETSNonSeasonal
[1] 2

$componentsNamesETS
[1] "level"    "trend"    "seasonal"

$obsStates
[1] 156

$modelIsTrendy
[1] TRUE

$modelIsSeasonal
[1] TRUE

$indexLookupTable
   [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
1     0    0    0    0    0    0    0    0    0     0     0     0     0     0
1     1    1    1    1    1    1    1    1    1     1     1     1     1     1
12    2    5    8   11   14   17   20   23   26    29    32    35     2     5
   [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
1      0     0     0     0     0     0     0     0     0     0     0     0
1      1     1     1     1     1     1     1     1     1     1     1     1
12     8    11    14    17    20    23    26    29    32    35     2     5
   [,27]

In [13]:
%%R -i ts_df
#library(smooth)

model <- adam(ts_df, model = "ZXZ", lags = c(12))

forecast(model, h = 12)


Error in attr(x, "tsp") <- c(1, NROW(x), 1) : 
  invalid time series parameters specified


RInterpreterError: Failed to parse and evaluate line '#library(smooth)\n\nmodel <- adam(ts_df, model = "ZXZ", lags = c(12))\n\nforecast(model, h = 12)\n'.
R error message: 'Error in attr(x, "tsp") <- c(1, NROW(x), 1) : \n  invalid time series parameters specified'

In [22]:
model = "ANN"
lags = [12]
multisteps = False,
lb = None,
ub = None,
maxtime = None,
print_level = 1, # 1 or 0
maxeval = None,
h = 12



# Assume that the model is not provided
# these will be default arguments
profiles_recent_provided = False
profiles_recent_table = None


In [ ]:
import time

start_time = time.time()
adam = Adam(model, lags)
adam.fit(ts_df, h = h)
fc = adam.predict()
execution_time = time.time() - start_time
print(f"Execution time: {execution_time:.4f} seconds")
fc['forecast']

In [10]:
adam.adam_estimated

{'B': array([ 1.        , 64.99718964]),
 'CF_value': 134.80985033320076,
 'n_param_estimated': 2,
 'log_lik_adam_value': {'value': np.float64(-134.80985033320076),
  'nobs': 24,
  'df': 3},
 'arima_polynomials': None}

In [11]:
fc['initial_estimated']

[True]